In [8]:
import pandas as pd
import datetime
import os
import timeit
from ortools.linear_solver import pywraplp

# Read Excel file
file_name = os.fsdecode("C:/Users/Administrator/VSCODE/BatteryOptimization/Data/Dataset_Optimization2.xlsx")
workbook = pd.ExcelFile(file_name)

output_folder = "output"
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)

# Load timeseries data
marketDF = workbook.parse("Timeseries data")
marketDF = marketDF.iloc[:,:5]  # Only read first 5 columns
marketDF.columns = ["time", "solar", "load", "market_price_1","feed_in_tariff"]
marketDF = marketDF[~pd.isnull(marketDF["time"])].fillna(0)

marketDF.sort_values(by=["time"], inplace=True)
marketDF.set_index("time", inplace=True)


# Load battery data
battDF = workbook.parse("Battery")
battDF = battDF.iloc[:,:8]
battDF.columns = ["max_charge_rate", "max_discharge_rate", "capacity", "charge_eff", "discharge_eff", "min_soc", "max_soc", "initial_soc"]

# Convert dataframe to dictionary
marketDict = marketDF.to_dict()

battDict = battDF.to_dict()

timeIndex = marketDF.index
timeInterval = timeIndex[1] - timeIndex[0]

# Assign the data to right places
# Set up input structure
input = type("input", (dict,), {})()
input.update({
    "simData": {
        "startTime": timeIndex[0],
        "dt": int(round(timeInterval.total_seconds())) / 3600,  # in hours
        "tIndex": len(timeIndex)
    },
    "market": {
        key: {
            sub_key: sub_item for sub_key, sub_item in marketDict[key].items()
        } for key in marketDict.keys()
    },
    "batt": {
        key: item[0] for key, item in battDict.items()
    }
})
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver.CreateSolver("CBC")

inf = solver.infinity()

tIndex = input["simData"]["tIndex"] # number of timeslots
dt = input["simData"]["dt"] # time interval in hour

# Create datetime array
startTime = input["simData"]["startTime"]
tIndex = input["simData"]["tIndex"]
timestamp = pd.date_range(start=startTime, periods=tIndex, freq=pd.Timedelta(hours=dt))
time = list(timestamp)

time_s = timeit.default_timer()
# Add timeseries variables
vGrid = [solver.NumVar(lb=-inf, ub=inf, name="") for _ in range(tIndex)]

vImport = [solver.NumVar(lb=0, ub=inf, name=f"import_{i}") for i in range(tIndex)]
vExport = [solver.NumVar(lb=0, ub=inf, name=f"export_{i}") for i in range(tIndex)]

vPV_export = [solver.NumVar(lb=0, ub=inf, name=f"pv_export_{i}") for i in range(tIndex)]

vBattPower = [solver.NumVar(lb=-inf, ub=inf, name="") for _ in range(tIndex)]
vCharge = [solver.NumVar(lb=-inf, ub=0, name="") for _ in range(tIndex)]
vDischarge = [solver.NumVar(lb=0, ub=inf, name="") for _ in range(tIndex)]
vChargeStatus = [solver.BoolVar(name="") for _ in range(tIndex)]
vSOC = [solver.NumVar(lb=0, ub=1, name="") for _ in range(tIndex)]

# Add constraints
for i in range(tIndex):
    t = time[i]

    # Grid balance constraint
    solver.Add(vGrid[i] == input["market"]["load"][t] - input["market"]["solar"][t] - vBattPower[i] + vPV_export[i])

    # Limit PV export to available solar
    solver.Add(vPV_export[i] <= input["market"]["solar"][t])

    # No battery export to grid
    solver.Add(vExport[i] >= vPV_export[i])

    # Grid constraints
    solver.Add(vGrid[i] == vImport[i] - vExport[i])


    # Battery constraints
    solver.Add(vBattPower[i] == vCharge[i] + vDischarge[i])
    solver.Add(vCharge[i] >= -input["batt"]["max_charge_rate"] * vChargeStatus[i])
    solver.Add(vDischarge[i] <= input["batt"]["max_discharge_rate"] * (1 - vChargeStatus[i]))

    if i == 0:
        solver.Add(vSOC[i] == input["batt"]["initial_soc"] - dt / input["batt"]["capacity"] * (vCharge[i] * (1 - input["batt"]["charge_eff"]) + vDischarge[i] / (1 - input["batt"]["discharge_eff"])))
    else:
        solver.Add(vSOC[i] == vSOC[i-1] - dt / input["batt"]["capacity"] * (vCharge[i] * (1 - input["batt"]["charge_eff"]) + vDischarge[i] / (1 - input["batt"]["discharge_eff"])))

    solver.Add(vSOC[i] >= input["batt"]["min_soc"])
    solver.Add(vSOC[i] <= input["batt"]["max_soc"])

# Add objective
obj = 0
obj += sum([
    vImport[i] * input["market"]["market_price_1"][time[i]] * dt -
    vPV_export[i] * input["market"]["feed_in_tariff"][time[i]] * dt
    for i in range(tIndex)
])
solver.Minimize(obj)

status = solver.Solve()

time_e = timeit.default_timer()
runTime = round(time_e - time_s, 4)

if status == solver.OPTIMAL or status == solver.FEASIBLE:
    print("Solution is found.")
    print("Number of variables =", solver.NumVariables())
    print("Number of constraints =", solver.NumConstraints())
    print("Computation time = ", runTime)

    # Extract solution values
    excelWriter = pd.ExcelWriter('output/Result2.xlsx', engine='xlsxwriter')

    # Zeitspezifische Werte berechnen
    import_cost = [vImport[i].solution_value() * input["market"]["market_price_1"][time[i]] * dt for i in range(tIndex)]
    feed_in_revenue = [vPV_export[i].solution_value() * input["market"]["feed_in_tariff"][time[i]] * dt for i in range(tIndex)]
    net_cost = [import_cost[i] - feed_in_revenue[i] for i in range(tIndex)]

    result = list(zip(
        [round(vGrid[i].solution_value(), 4) for i in range(tIndex)], 
        [round(vImport[i].solution_value(), 4) for i in range(tIndex)],
        [round(vExport[i].solution_value(), 4) for i in range(tIndex)],
        [round(vPV_export[i].solution_value(), 4) for i in range(tIndex)],
        [round(vBattPower[i].solution_value(), 4) for i in range(tIndex)],
        [round(vCharge[i].solution_value(), 4) for i in range(tIndex)],
        [round(vDischarge[i].solution_value(), 4) for i in range(tIndex)],
        [round(vSOC[i].solution_value(), 4) for i in range(tIndex)],
        [int(vChargeStatus[i].solution_value()) for i in range(tIndex)],
        [round(import_cost[i], 4) for i in range(tIndex)],
        [round(feed_in_revenue[i], 4) for i in range(tIndex)],
        [round(net_cost[i], 4) for i in range(tIndex)]
    ))

    resultDF = pd.DataFrame(result, index=timestamp, columns=[
        "Grid Power Flow (kW)", 
        "Import (kW)", 
        "Export (kW)", 
        "PV Export (kW)", 
        "Battery Output (kW)", 
        "Charging Power (kW)", 
        "Discharging Power (kW)", 
        "State-of-charge (SOC)", 
        "Charge Status",
        "Import Cost ($)",
        "Feed-in Revenue ($)",
        "Net Cost ($)"
    ])

    # Gesamtkosten
    total_import_cost = round(sum(import_cost), 2)
    total_revenue = round(sum(feed_in_revenue), 2)
    net_total_cost = round(total_import_cost - total_revenue, 2)

    cashflowDF = pd.DataFrame.from_dict({
        "Total Import Cost ($)": [total_import_cost],
        "Total Feed-in Revenue ($)": [total_revenue],
        "Net Cost ($)": [net_total_cost]
    }, orient="index", columns=["Total ($)"])

    # Output schreiben
    resultDF.to_excel(excelWriter, sheet_name='Operation')
    cashflowDF.to_excel(excelWriter, sheet_name='Cost Summary')
    excelWriter.close()
else:
    print("Solution cannot be found.")


Solution is found.
Number of variables = 6696
Number of constraints = 7440
Computation time =  1.4673
